In [6]:
# usual libraries
import numpy as np
import matplotlib.pyplot as plt
import functools
from tqdm import tqdm, trange

# Wolff Algorithm for PBC 1D Ising Chain
def wolffAlgorithm(chainLattice, K, steps):
    size = np.size(chainLattice)

    for s in range(steps):
        cluster = wolffStep(chainLattice, K)
        # flip the cluster
        for site in cluster:
            chainLattice[site] *= -1

def wolffStep(chainLattice, K):
    size = np.size(chainLattice)
    pAdd = 1 - np.exp(-2*K)
    # begin wolff step
    idx = np.random.randint(size)

    # initialize frontier and cluster
    frontier = {idx}
    cluster  = {idx}

    # expand the cluster
    while len(frontier) > 0:

        newFrontier = set()

        for site in frontier:

            # check if sites to left and right are to be added
            nNbr = (site + 1) % size
            if (chainLattice[site] == chainLattice[nNbr] and
                    nNbr not in cluster and
                    np.random.rand() < pAdd):

                newFrontier.add(nNbr)
                cluster.add(nNbr)
            pNbr = (site - 1) % size
            if (chainLattice[site] == chainLattice[pNbr] and
                    pNbr not in cluster and
                    np.random.rand() < pAdd):

                newFrontier.add(pNbr)
                cluster.add(pNbr)

        frontier = newFrontier.copy()

    return cluster

def getSamples(chainLattice, K, sampleSteps, equilTime=1500, autoCorrTime=1500):

    size = np.size(chainLattice)
    sampleArray = np.zeros((sampleSteps, size))
    for s in range(round(equilTime)):
        cluster = wolffStep(chainLattice, K)

        for site in cluster:
            chainLattice[site] *= -1

    for s in trange(sampleSteps):

        for ss in trange(round(autoCorrTime)):
            cluster = wolffStep(chainLattice, K)
            for site in cluster:
                chainLattice[site] *= -1

        sampleArray[s] = chainLattice

    return sampleArray

def getMeasurements(chainLattice, K, sampleSteps, observable, autoCorrTime=None):
    size = np.size(chainLattice)
    obsArray = np.zeros(sampleSteps)
    if autoCorrTime == None:
        autoCorrTime = round(1.5*size)
        equilTime = autoCorrTime

    for s in range(equilTime):
        cluster = wolffStep(chainLattice, K)

        for site in cluster:
            chainLattice[site] *= -1

    for s in range(sampleSteps):

        for ss in range(autoCorrTime):
            cluster = wolffStep(chainLattice, K)
            for site in cluster:
                chainLattice[site] *= -1

        obsArray[s] = observable(chainLattice, K)

    return obsArray
# observables
def energy(chainLattice, K):
    size = np.size(chainLattice)
    E = 0

    for site in range(size):
        alignSgn = chainLattice[site]*chainLattice[(site+1)%size]
        E += -2*K*alignSgn

    return E

def corrAtR(chainLattice, K, r):
    size = np.size(chainLattice)
    r = r % size

    return chainLattice[0]*chainLattice[r]

def corrFunc(chainLattice, K):
    size = np.size(chainLattice)
    maxR = round(size/2)

    corrFuncArr = np.zeros(maxR)

    for r, _ in enumerate(corrFuncArr):
        corrFuncArr[r] = corrAtR(chainLattice, K, r)

    return corrFuncArr


# In[5]:


def getCorrelations(chainLattice, K, sampleSteps, autoCorrTime=None):
    size = np.size(chainLattice)
    maxR = round(size/2)
    corrArray = np.zeros((sampleSteps, maxR))
    if autoCorrTime == None:
        autoCorrTime = round(1.5*size)
        equilTime = autoCorrTime

    for s in range(equilTime):
        cluster = wolffStep(chainLattice, K)

        for site in cluster:
            chainLattice[site] *= -1

    for s in range(sampleSteps):
        for ss in range(autoCorrTime):
            cluster = wolffStep(chainLattice, K)

            for site in cluster:
                chainLattice[site] *= -1

        corrArray[s] = corrFunc(chainLattice, K)

    dataArray = np.stack(
        (np.average(corrArray, axis=0), np.std(corrArray, axis=0)/np.sqrt(sampleSteps))
    )
    xdata = np.arange(maxR)
    return (xdata, dataArray)


# In[6]:


# some wrappers
def timethis(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(func.__name__, end-start)
        return result
    return wrapper


In [12]:
range_K = np.arange(9) + 2

In [14]:
for K in range_K:
    print("Doing K=", K)
    sampleArray = getSamples(chainLattice, K=K, sampleSteps=1000)
    np.savez(f"1dNNMC_K={K}.npz", data=sampleArray)

Doing K= 2


100%|██████████| 1000/1000 [01:20<00:00, 12.39it/s]


Doing K= 3


100%|██████████| 1000/1000 [08:51<00:00,  1.88it/s]


Doing K= 4


 20%|██        | 203/1000 [04:10<16:24,  1.24s/it]


KeyboardInterrupt: 

In [15]:
a = np.load("1dNNMC_K=3.npz")["data"]